## GGjets parquet reading
Reading of events and convert it to a root file as histogram and as trees for all regions and write it to root 

In [23]:
import os
import uproot
import pandas as pd


from config.utils import lVector, VarToHist
from normalisation import getXsec, getLumi


Welcome to JupyROOT 6.30/02


In [ ]:
# df = pd.read_parquet("../output_root/v2_production_central/backgrounds/GGJets.root")

In [1]:
# import pyarrow.parquet as pq

# # Read the Parquet file
# table = pq.read_table("../output_root/v2_production_central/backgrounds/GGJets.root")

# # Convert to pandas DataFrame
# df = table.to_pandas()
# print(df.head())


In [4]:
import pyarrow.parquet as pq

# Open file
parquet_file = pq.ParquetFile("../output_root/v2_production_central/backgrounds/GGJets.root")

# Read in chunks
for batch in parquet_file.iter_batches(batch_size=100000):
    df = batch.to_pandas()
    print(df.head())  # Process batch
    break  # Remove this to process all batches


   lead_seediEtaOriX  lead_cutBased  lead_electronVeto  \
0                 -7              1               True   
1                 48              3               True   
2                -43              1               True   
3                 45              3               True   
4                 77              3               True   

   lead_hasConversionTracks  lead_isScEtaEB  lead_isScEtaEE  lead_mvaID_WP80  \
0                     False            True           False            False   
1                     False            True           False             True   
2                     False            True           False             True   
3                     False            True           False             True   
4                     False           False            True             True   

   lead_mvaID_WP90  lead_pixelSeed  lead_seedGain  ...  \
0            False           False             12  ...   
1             True           False             12  ...

In [13]:
required_columns = [
        "run", "lumi", "event",
        "puppiMET_pt",
            "puppiMET_phi",
            "puppiMET_phiJERDown",
            "puppiMET_phiJERUp",
            "puppiMET_phiJESDown",
            "puppiMET_phiJESUp",
            "puppiMET_phiUnclusteredDown",
            "puppiMET_phiUnclusteredUp",
            "puppiMET_ptJERDown",
            "puppiMET_ptJERUp",
            "puppiMET_ptJESDown",
            "puppiMET_ptJESUp",
            "puppiMET_ptUnclusteredDown",
            "puppiMET_ptUnclusteredUp",
            "puppiMET_sumEt",
            "Res_lead_bjet_pt",
            "Res_lead_bjet_eta",
            "Res_lead_bjet_phi",
            "Res_lead_bjet_mass",
            "Res_sublead_bjet_pt",
            "Res_sublead_bjet_eta",
            "Res_sublead_bjet_phi",
            "Res_sublead_bjet_mass",
            "lead_pt",
            "lead_eta",
            "lead_phi",
            "lead_mvaID_WP90",
            "lead_mvaID_WP80",  # tight PhotonID? or is it loose?
            "sublead_pt",
            "sublead_eta",
            "sublead_phi",
            "sublead_mvaID_WP90",
            "sublead_mvaID_WP80",
            "weight",
            "weight_central",
            "Res_lead_bjet_btagPNetB",
            "Res_sublead_bjet_btagPNetB",
            "lead_isScEtaEB",
            "sublead_isScEtaEB",
            "Res_HHbbggCandidate_pt",
            "Res_HHbbggCandidate_eta",
            "Res_HHbbggCandidate_phi",
            "Res_HHbbggCandidate_mass",
            "Res_CosThetaStar_CS",
            "Res_CosThetaStar_gg",
            "Res_CosThetaStar_jj",
            "Res_DeltaR_jg_min",
            "Res_pholead_PtOverM",
            "Res_phosublead_PtOverM",
            "Res_FirstJet_PtOverM",
            "Res_SecondJet_PtOverM",
            "lead_mvaID",
            "sublead_mvaID",
            "Res_DeltaR_j1g1",
            "Res_DeltaR_j2g1",
            "Res_DeltaR_j1g2",
            "Res_DeltaR_j2g2",
            "Res_M_X",
            ## Adding other variables
            # "pholead_PtOverM",
            # "phosublead_PtOverM",
            # "FirstJet_PtOverM",
            # "SecondJet_PtOverM",
            # "CosThetaStar_CS",
            # "CosThetaStar_gg",
            # "CosThetaStar_jj",
            # "DeltaR_j1g1",
            # "DeltaR_j2g1",
            # "DeltaR_j1g2",
            # "DeltaR_j2g2",
            # "DeltaR_j2g2",
    ]


# Reading parquet

In [27]:
# Sample input file
inputfile = "../output_root/v2_production_central/backgrounds/GGJets.parquet"

# Initialize flags
isdata = False
isSignal = False

# Determine if it's Data or Signal
filename = inputfile.split("/")[-1]

if "Data" in filename:
    isdata = True
    xsec_ = 1
    lumi_ = 1
else:
    xsec_ = getXsec(inputfile)  # Fetch cross-section
    lumi_ = getLumi() * 1000    # Convert luminosity

if "GluGluToHH" in filename:
    isSignal = True

# Print status
print("Status of the isdata flag:", isdata)
print("Status of the isSignal flag:", isSignal)
print("Cross-section:", xsec_)
print("Luminosity:", lumi_)


Extracted sample name: '../output_root/v2_production_central/backgrounds/GGJets.parquet'
Status of the isdata flag: False
Status of the isSignal flag: False
Cross-section: 88.75
Luminosity: 26671.7


In [28]:
import pandas as pd

# Define required columns
# required_columns = ["col1", "col2", "col3"]  # Adjust as needed

# Read specific columns with pyarrow for better performance
df = pd.read_parquet(
    inputfile,
    columns=required_columns,  
    engine="pyarrow"
)

# Display first few rows
# print(df)
print(df.info())  # Shows column types & memory usage


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255690 entries, 0 to 3255689
Data columns (total 61 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   run                          uint32 
 1   lumi                         uint32 
 2   event                        uint64 
 3   puppiMET_pt                  float32
 4   puppiMET_phi                 float32
 5   puppiMET_phiJERDown          float32
 6   puppiMET_phiJERUp            float32
 7   puppiMET_phiJESDown          float32
 8   puppiMET_phiJESUp            float32
 9   puppiMET_phiUnclusteredDown  float32
 10  puppiMET_phiUnclusteredUp    float32
 11  puppiMET_ptJERDown           float32
 12  puppiMET_ptJERUp             float32
 13  puppiMET_ptJESDown           float32
 14  puppiMET_ptJESUp             float32
 15  puppiMET_ptUnclusteredDown   float32
 16  puppiMET_ptUnclusteredUp     float32
 17  puppiMET_sumEt               float32
 18  Res_lead_bjet_pt             float64
 19  

In [36]:
import pandas as pd

# Define the mapping of old column names to new column names
cms_events = {
    "run": "run",
    "lumi": "lumi",
    "event": "event",
    "puppiMET_pt": "puppiMET_pt",
    "puppiMET_phi": "puppiMET_phi",
    "puppiMET_phiJERDown": "puppiMET_phiJERDown",
    "puppiMET_phiJERUp": "puppiMET_phiJERUp",
    "puppiMET_phiJESDown": "puppiMET_phiJESDown",
    "puppiMET_phiJESUp": "puppiMET_phiJESUp",
    "puppiMET_phiUnclusteredDown": "puppiMET_phiUnclusteredDown",
    "puppiMET_phiUnclusteredUp": "puppiMET_phiUnclusteredUp",
    "puppiMET_ptJERDown": "puppiMET_ptJERDown",
    "puppiMET_ptJERUp": "puppiMET_ptJERUp",
    "puppiMET_ptJESDown": "puppiMET_ptJESDown",
    "puppiMET_ptJESUp": "puppiMET_ptJESUp",
    "puppiMET_ptUnclusteredDown": "puppiMET_ptUnclusteredDown",
    "puppiMET_ptUnclusteredUp": "puppiMET_ptUnclusteredUp",
    "puppiMET_sumEt": "puppiMET_sumEt",
    "Res_lead_bjet_pt": "lead_bjet_pt",
    "Res_lead_bjet_eta": "lead_bjet_eta",
    "Res_lead_bjet_phi": "lead_bjet_phi",
    "Res_lead_bjet_mass": "lead_bjet_mass",
    "Res_sublead_bjet_pt": "sublead_bjet_pt",
    "Res_sublead_bjet_eta": "sublead_bjet_eta",
    "Res_sublead_bjet_phi": "sublead_bjet_phi",
    "Res_sublead_bjet_mass": "sublead_bjet_mass",
    "lead_pt": "lead_pho_pt",
    "lead_eta": "lead_pho_eta",
    "lead_phi": "lead_pho_phi",
    "lead_mvaID_WP90": "lead_pho_mvaID_WP90",
    "lead_mvaID_WP80": "lead_pho_mvaID_WP80",
    "sublead_pt": "sublead_pho_pt",
    "sublead_eta": "sublead_pho_eta",
    "sublead_phi": "sublead_pho_phi",
    "sublead_mvaID_WP90": "sublead_pho_mvaID_WP90",
    "sublead_mvaID_WP80": "sublead_pho_mvaID_WP80",
    "weight_central": "weight_central",
    "weight": "weight",
    "Res_lead_bjet_btagPNetB": "lead_bjet_PNetB",
    "Res_sublead_bjet_btagPNetB": "sublead_bjet_PNetB",
    "lead_isScEtaEB": "lead_isScEtaEB",
    "sublead_isScEtaEB": "sublead_isScEtaEB",
    "Res_CosThetaStar_CS": "CosThetaStar_CS",
    "Res_CosThetaStar_gg": "CosThetaStar_gg",
    "Res_CosThetaStar_jj": "CosThetaStar_jj",
    "Res_DeltaR_jg_min": "DeltaR_jg_min",
    "Res_pholead_PtOverM": "pholead_PtOverM",
    "Res_phosublead_PtOverM": "phosublead_PtOverM",
    "Res_FirstJet_PtOverM": "FirstJet_PtOverM",
    "Res_SecondJet_PtOverM": "SecondJet_PtOverM",
    "lead_mvaID": "lead_pho_mvaID",
    "sublead_mvaID": "sublead_pho_mvaID",
    "Res_DeltaR_j1g1": "DeltaR_j1g1",
    "Res_DeltaR_j2g1": "DeltaR_j2g1",
    "Res_DeltaR_j1g2": "DeltaR_j1g2",
    "Res_DeltaR_j2g2": "DeltaR_j2g2",
    "Res_HHbbggCandidate_mass": "bbgg_mass",
    "Res_HHbbggCandidate_pt": "bbgg_pt",
    "Res_HHbbggCandidate_eta": "bbgg_eta",
    "Res_HHbbggCandidate_phi": "bbgg_phi",
    "Res_M_X": "MX",
}

# Apply renaming
df = df.rename(columns=cms_events)

# Display first few rows to verify renaming
print(df.head())


   run     lumi  event  puppiMET_pt  puppiMET_phi  puppiMET_phiJERDown  \
0    1  26671.7   8457    28.794365     -2.702637            -2.724609   
1    1  26671.7   8485    28.510080     -1.231934            -1.179688   
2    1  26671.7   8508    20.095448      1.424805             0.713379   
3    1  26671.7   8522    19.977036     -0.033577            -0.047729   
4    1  26671.7   8520    41.458942      2.011719             2.269531   

   puppiMET_phiJERUp  puppiMET_phiJESDown  puppiMET_phiJESUp  \
0          -2.681641            -2.650391          -2.748047   
1          -1.241211            -1.186523          -1.274414   
2           0.666016             1.394531           1.454102   
3          -0.021072            -0.057465          -0.012848   
4           1.870117             2.027344           1.996094   

   puppiMET_phiUnclusteredDown  ...  lead_pho_mvaID  sublead_pho_mvaID  \
0                    -2.712891  ...       -0.200832          -0.185993   
1                    -

In [19]:
df

,run,lumi,event,puppiMET_pt,puppiMET_phi,puppiMET_phiJERDown,puppiMET_phiJERUp,puppiMET_phiJESDown,puppiMET_phiJESUp,puppiMET_phiUnclusteredDown,...,phosublead_PtOverM,FirstJet_PtOverM,SecondJet_PtOverM,lead_pho_mvaID,sublead_pho_mvaID,DeltaR_j1g1,DeltaR_j2g1,DeltaR_j1g2,DeltaR_j2g2,MX
0,1,13,8457,28.794365,-2.702637,-2.724609,-2.681641,-2.650391,-2.748047,-2.712891,...,0.339349,0.635543,0.339816,-0.200832,-0.185993,2.465026,0.751889,1.904712,2.989284,288.383802
1,1,13,8485,28.510080,-1.231934,-1.179688,-1.241211,-1.186523,-1.274414,-1.235352,...,0.377239,1.202877,1.199028,0.857354,0.950449,2.613601,2.654957,2.269721,2.535428,292.747212
2,1,13,8508,20.095448,1.424805,0.713379,0.666016,1.394531,1.454102,1.535156,...,0.363359,0.381669,0.256452,0.811463,-0.421172,2.754449,3.264950,1.787065,1.271481,269.310092
3,1,13,8522,19.977036,-0.033577,-0.047729,-0.021072,-0.057465,-0.012848,0.006325,...,0.461090,0.965347,0.462599,0.882302,0.829067,3.238603,1.667862,1.733435,2.828018,395.196128
4,1,13,8520,41.458942,2.011719,2.269531,1.870117,2.027344,1.996094,1.955078,...,0.388469,1.415862,0.267875,0.954967,0.912090,2.757194,2.076575,0.651147,1.355291,295.148769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255685,1,64143,45156618,35.837994,-1.710938,-1.682617,-1.736328,-1.721680,-1.700195,-1.705078,...,0.449013,0.743418,0.421236,0.808635,0.945792,3.162483,1.412506,1.810470,1.447131,285.850530
3255686,1,64143,45156647,17.417778,-3.109375,-3.017578,3.097656,-2.892578,2.992188,3.128906,...,0.334596,0.786489,0.280671,0.746193,0.939006,3.028928,1.686959,2.727265,1.561175,325.421573
3255687,1,64143,45156643,5.215833,0.207062,0.811035,-0.172729,0.234985,0.180908,0.264648,...,0.391613,0.589927,0.410526,0.947468,0.584231,2.288928,2.256603,1.559899,1.678578,278.065669
3255688,1,64143,45156649,75.104446,-1.932129,-1.866211,-1.846680,-1.944336,-1.921875,-1.924805,...,0.476791,0.642182,0.149672,0.873534,0.486540,2.414054,1.822329,2.765797,0.695387,334.804351


In [21]:
out_events = df[["run", "lumi", "event"]].copy()


In [22]:
out_events

,run,lumi,event
0,1,13,8457
1,1,13,8485
2,1,13,8508
3,1,13,8522
4,1,13,8520
...,...,...,...
3255685,1,64143,45156618
3255686,1,64143,45156647
3255687,1,64143,45156643
3255688,1,64143,45156649


In [38]:
df["isdata"] = isdata
df["isSignal"] = isSignal
df["xsec"] = xsec_
df["lumi"] = lumi_

# Display to verify
print(df.head())


   run     lumi  event  puppiMET_pt  puppiMET_phi  puppiMET_phiJERDown  \
0    1  26671.7   8457    28.794365     -2.702637            -2.724609   
1    1  26671.7   8485    28.510080     -1.231934            -1.179688   
2    1  26671.7   8508    20.095448      1.424805             0.713379   
3    1  26671.7   8522    19.977036     -0.033577            -0.047729   
4    1  26671.7   8520    41.458942      2.011719             2.269531   

   puppiMET_phiJERUp  puppiMET_phiJESDown  puppiMET_phiJESUp  \
0          -2.681641            -2.650391          -2.748047   
1          -1.241211            -1.186523          -1.274414   
2           0.666016             1.394531           1.454102   
3          -0.021072            -0.057465          -0.012848   
4           1.870117             2.027344           1.996094   

   puppiMET_phiUnclusteredDown  ...  lead_pho_mvaID  sublead_pho_mvaID  \
0                    -2.712891  ...       -0.200832          -0.185993   
1                    -

In [45]:
# dibjet_ = lVector(
#     df["Res_lead_bjet_pt"],
#     df["lead_bjet_eta"],
#     df["lead_bjet_phi"],
#     df["sublead_bjet_pt"],
#     df["sublead_bjet_eta"],
#     df["sublead_bjet_phi"],
#     df["lead_bjet_mass"],
#     df["sublead_bjet_mass"],
#         )
# diphoton_ = lVector(
#     df["lead_pho_pt"],
#     df["lead_pho_eta"],
#     df["lead_pho_phi"],
#     df["sublead_pho_pt"],
#     df["sublead_pho_eta"],
#     df["sublead_pho_phi"],
#         )

In [48]:
from regions import (
            get_mask_preselection,
            get_mask_selection,
            get_mask_srbbgg,
            get_mask_srbbggMET,
            get_mask_crantibbgg,
            get_mask_crbbantigg,
        )